# Tugas IF4070 - Representasi Pengetahuan dan Penalaran
# Implementasi Ripple Down Rules

# Simple RDR
**Simple RDR** is a ripple down rules implemented in Python.

## Setup
Assuming you've installed the latest version of Python (if not, guides for it are widely available),
1. ensure pip is installed by running `python -m ensurepip --upgrade`;
2. install the Python dependencies by running `pip install -r requirements.txt`.

In [1]:
import random

In [2]:
class Node:
    def __init__(
        self,
        precedent: str,
        antecedent: str,
        cornerstone: set[str],
        except_: "Node" = None,
        else_: "Node" = None,
        is_root: bool = False,
    ) -> None:
        self._precedent = precedent
        self._antecedent = antecedent
        self._cornerstone = cornerstone
        self._except = except_
        self._else = else_
        self._is_root = is_root

    def get_antecedent(self) -> str:
        return self._antecedent

    def get_cornerstone(self) -> set[str]:
        return self._cornerstone

    def get_except(self) -> "Node":
        return self._except

    def get_else(self) -> "Node":
        return self._else

    def set_except(self, except_: "Node") -> None:
        self._except = except_

    def set_else(self, else_: "Node") -> None:
        self._else = else_

    def match_precedent(self, case: set[str]) -> bool:
        if self._is_root:
            return True
        else:
            for statement in case:
                if statement.startswith("~"):
                    exec(f"{statement[1:]} = False")
                else:
                    exec(f"{statement} = True")

            try:
                eval(self._precedent)
                return True
            except NameError:
                return False
            finally:
                for statement in case:
                    if statement.startswith("~"):
                        exec(f"del {statement[1:]}")
                    else:
                        exec(f"del {statement}")

In [3]:
class Tree:
    def __init__(self, root: "Node"):
        self._root = root

    def _traverse_tree(self, case: set[str]) -> None:
        current_node = self._root
        last_true = self._root

        while current_node and (current_node.get_except() or current_node.get_else()):
            if current_node.match_precedent(case):
                last_true = current_node
                current_node = current_node.get_except()
            else:
                current_node = current_node.get_else()
        else:
            if next_node := current_node.match_precedent(case):
                last_true = current_node

        antecedent = last_true.get_antecedent()
        print(f"The system concludes that your case can be associated with the following: {antecedent}.")
        print("Do you agree? (y/n)")

        while True:
            agreement = input(
                f"The system concludes that your case can be associated with the following: {antecedent}.\nDo you agree? (y/n)"
            ).lower()
            print(f"You entered: {agreement}")
            if agreement in ("y", "n"):
                break
            else:
                print(f"Please input a valid option!")

        if agreement == "n":
            print("Please input a correct conclusion for this case!")
            conclusion = input("Please input a correct conclusion for this case!")
            print(f"You entered: {conclusion}")

            new_precedent = random.choice(tuple(case - last_true.get_cornerstone()))
            new_node = Node(new_precedent, conclusion, case)
            if next_node:
                current_node.set_except(new_node)
            else:
                current_node.set_else(new_node)

    def start(self) -> None:
        print("Welcome to RDR Expert System!")
        print()

        while True:
            print("Enter your case, separated by a comma for each fact!")
            print("Example case: mammal, fly, ~swim")
            print("Input your case!")

            case = set(input("Input your case: ").split(", "))
            print(f"You entered: {case}")
            self._traverse_tree(case)

            while True:
                print()
                print("Would you like to evaluate a different case? (y/n)")
                continue_use = input("Would you like to evaluate a different case? (y/n) ").lower()
                if continue_use in ("y", "n"):
                    break
                else:
                    print("Please input a valid option!")

            if continue_use == "y":
                continue
            else:
                break

In [4]:
root_node = Node("", "human", set(), is_root=True)
model = Tree(root_node)

model.start()

Welcome to RDR Expert System!

Enter your case, separated by a comma for each fact!
Example case: mammal, fly, ~swim
Input your case!
You entered: {'swim'}
The system concludes that your case can be associated with the following: human.
Do you agree? (y/n)
You entered: n
Please input a correct conclusion for this case!
You entered: fish

Would you like to evaluate a different case? (y/n)
Enter your case, separated by a comma for each fact!
Example case: mammal, fly, ~swim
Input your case!
You entered: {'swim'}
The system concludes that your case can be associated with the following: fish.
Do you agree? (y/n)
You entered: y

Would you like to evaluate a different case? (y/n)
Enter your case, separated by a comma for each fact!
Example case: mammal, fly, ~swim
Input your case!
You entered: {'mammal', 'swim'}
The system concludes that your case can be associated with the following: fish.
Do you agree? (y/n)
You entered: n
Please input a correct conclusion for this case!
You entered: dolph